In [ ]:
%load_ext autoreload
%autoreload 2

import os
import re
import json
import glob
import random

import sys
sys.path.append('simulation')
from unity_simulator.comm_unity import UnityCommunication
from dataset_utils import execute_script_utils as utils
from evolving_graph import scripts

def setup():
    comm = UnityCommunication()
    return comm

### Fetching all scripts and running them to filter the ones that run on sim

In [ ]:
DATASET = 'dataset/programs_processed'
list_of_scene_scripts = glob.glob('{}/executable_programs/TrimmedTestScene{}_graph/*/*.txt'.format(DATASET,'1'))
def graph_file_from_script(script_file):
    return script_file.replace('executable_programs','init_and_final_graphs').replace('.txt','.json')

In [ ]:
# # These were found to get the simulator stuck...
list_of_scene_scripts.remove('dataset/programs_processed/executable_programs/TrimmedTestScene1_graph/results_text_rebuttal_specialparsed_programs_turk_third/split36_2.txt')
list_of_scene_scripts.remove('dataset/programs_processed/executable_programs/TrimmedTestScene1_graph/results_intentions_march-13-18/file241_1.txt')
list_of_scene_scripts.remove('dataset/programs_processed/executable_programs/TrimmedTestScene1_graph/results_intentions_march-13-18/file416_2.txt')
list_of_scene_scripts.remove('dataset/programs_processed/executable_programs/TrimmedTestScene1_graph/results_intentions_march-13-18/file211_1.txt')
list_of_scene_scripts.remove('dataset/programs_processed/executable_programs/TrimmedTestScene1_graph/results_intentions_march-13-18/file474_2.txt')
list_of_scene_scripts.remove('dataset/programs_processed/executable_programs/TrimmedTestScene1_graph/results_intentions_march-13-18/file287_2.txt')
list_of_scene_scripts.remove('dataset/programs_processed/executable_programs/TrimmedTestScene1_graph/results_intentions_march-13-18/file99_2.txt')
list_of_scene_scripts.remove('dataset/programs_processed/executable_programs/TrimmedTestScene1_graph/results_intentions_march-13-18/file482_2.txt')
list_of_scene_scripts.remove('dataset/programs_processed/executable_programs/TrimmedTestScene1_graph/results_intentions_march-13-18/file352_1.txt')
list_of_scene_scripts.remove('dataset/programs_processed/executable_programs/TrimmedTestScene1_graph/results_intentions_march-13-18/file442_2.txt')
list_of_scene_scripts.remove('dataset/programs_processed/executable_programs/TrimmedTestScene1_graph/results_text_rebuttal_specialparsed_programs_upwork_july/split23_9.txt')


In [ ]:
def process_file_inbuilt(script_file, graph_file):
    return utils.render_script_from_path(setup(), 
                                  script_file, graph_file,
                                  {"processing_time_limit": 20, "image_width": 320, "image_height": 240, "image_synthesis": ['normal'], "gen_vid": True, "file_name_prefix": "test"})

print('Total scripts to go through : ',len(list_of_scene_scripts))
usable_scripts = []
try:
    for i,script_file in enumerate(list_of_scene_scripts):
        res = process_file_inbuilt(script_file, graph_file_from_script(script_file))
        success = res['success_expand']
        if 'success_exec' in res.keys():
            success = success and res['success_exec']
        if success:
            usable_scripts.append(script_file)
        if i%50 == 0:
            print(f'Processed {i} scripts')
except Exception as e:
    print(e)
finally:
    print(i,' files processed')
    print('Last processed :',script_file)
    print(len(usable_scripts))
    with open(DATASET+'/usable_scripts.txt','a') as f:
        for s in usable_scripts:
            f.write(s+'\n')

### Categorize the usable script by activity and save in short_scripts

In [ ]:
print(len(usable_scripts))
if not usable_scripts:
    with open(DATASET+'/usable_scripts.txt','r') as f:
        for line in f:
            usable_scripts.append(line[:-1])
print(len(usable_scripts))

activities = {}
for script in usable_scripts:
    with open(script,'r') as f:
        activity = f.readline().strip()
        if activity in activities:
            activities[activity].append(script)
        else:
            activities[activity]=[script]
for act in activities:
    print (act,len(activities[act]))

### Fetching all scripts and running them to filter the ones that run on sim

In [ ]:
## Manual Work here!!!
activity_categories = {}
activity_categories['morning'] = ['Take shower', 'Turn on light', 'Open front door','Open door']
activity_categories['night'] = ['Turn off light', 'Walk to room', 'Lock door']
activity_categories['pastime'] = ['Read book', 'Read', 'Do work', 'Pet cat', 'Pick up phone', 'Go to toilet', 'Relax on sofa', 'Browse internet', 'Listen to music', 'Watch TV', 'Admire art']
activity_categories['chore'] = ['Change sheets and pillow cases', 'Find dictionary','Put away shoes']
activity_categories['food_pre'] = []
activity_categories['food_post'] = ['Bring dirty plate to sink', ]
activity_categories['unknown'] = ['Turn off TV', 'Bring me red cookbook', 'Close door', 'Keep cats out of room']

activity_scripts = {key:[] for key in activity_categories.keys()}

for act,scripts in activities.items():
    for k,v in activity_categories.items():
        if act in v:
            activity_scripts[k] = activity_scripts[k] + scripts


In [ ]:
from shutil import copyfile

for act, scripts in activity_scripts.items():
    os.makedirs('dataset/short_scripts/withoutconds/'+act)
    print(act, len(scripts))
    for script in scripts:
        script = script.replace('executable_programs/TrimmedTestScene1_graph','withoutconds')
        copyfile(script, 'dataset/short_scripts/withoutconds/'+act+'/'+os.path.basename(script))

### Generate concatenated scripts for full-day routine

In [ ]:
routine_components = ['morning','night','pastime','chore','food_post']
routine_sequence = ['morning','morning','chore','pastime','food_post','pastime','chore','food_post','pastime','night']

def write_script(script, file_name, script_name='', script_desc=''):
    with open(file_name, 'w') as f:
        f.write(script_name+'\n')
        f.write(script_desc+'\n')
        f.write('\n')
        for s in script:
            f.write(s+'\n')
 

def sample_script():
    script = []
    for segment in routine_sequence:
        file_name = random.choice(activity_scripts[segment])
        file_name = file_name.replace('executable_programs/TrimmedTestScene1_graph','withoutconds')
        with open(file_name) as f:
            for line in f:
                if '[' in line:
                    script.append(line.strip())
        script.append(' ')
    return script

for i in range(5):
    write_script(sample_script(), 'dataset/long_scripts/withoutconds/batch2/'+str(i)+'.txt', 'Randomly generated routine', str(routine_sequence))

In [ ]:
def parse_exec_script_file(file_name):
    with open(file_name, 'r') as f:
        content = f.readlines()
        content = [x.strip() for x in content]
        title = content[0]
        description = content[1]
        script_raw = content[4:]

    script = []
    for elem in script_raw:
        script.append(re.sub('[0-9]\.', '', elem))
    print(script)
    return title, description, script

def obtain_objects_from_message(message):
    objects_missing = []
    for x in ['unplaced', 'missing_destinations', 'missing_prefabs']:
        if x in message.keys():
            objects_missing += message[x]
    return objects_missing

def process_file_elaborate():
    # get comm
    comm = setup()
    status = comm.reset(0)
    if not status:
        print("Error: could not load sim.")
        exit()
    # get script
    try:
        title, desc, script = parse_exec_script_file(SCRIPT_FILE)
    except FileNotFoundError:
        print("Error: could not load script file.")
        exit()
    script_content = scripts.read_script_from_list_string(script)
    print("TITLE: " + title)
    print("DESCRIPTION: " + desc)
    print("### SCRIPT ###")
    print(script)
    print("### SCRIPT ###")
    # get init_graph
    graphs = None
    try:
        with open(GRAPH_FILE, "r") as f:
            graphs = json.load(f)
    except FileNotFoundError:
        print("Error: could not load graph file.")
        exit()
    init_graph = graphs["init_graph"]
    status, msg = comm.expand_scene(init_graph)
    if not status:
        # sometimes fails due to "background" not renderable by unity
        objects_missing = obtain_objects_from_message(msg)
        objects_script = [x[0].replace('_', '') for x in script_content.obtain_objects()]
        intersection_objects = list(set(objects_script).intersection(objects_missing))
        if len(intersection_objects) > 0:
            print("Error: could not load init_graph. See below.")
            print(msg)
            exit()
    # finally with comm, graph, and script, render the script
    render_args = {"processing_time_limit": 120,
                "image_width": 320,
                "image_height": 240,
                "image_synthesis": ['normal'],
                "gen_vid": True,
                "file_name_prefix": "test"}
    status, msg = comm.render_script(script, **render_args)
    if status:
        print("robotHow recipe executed!")
    else:
        print(msg)

Scripts now run on their own init graph. How to generate an init graph for a sequence? Complete and fix the scripts manually then concatenate and try creating init graph and executable